In [1]:
import requests
import pandas as pd
#quiero mucho a uli



In [2]:
def get_cycles_data(curp):
    BASE_CURP_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/schoolCyclesByStudent/curp={curp}"
    response = requests.get(BASE_CURP_URL,verify=False)
    if response.status_code == 200:
        return response.json()
    print(response.status_code)
    return None


def get_grades_data(StudentCycleID):
    BASE_GRADES_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/evaluationsByStudentCycle/idStudentCycle={StudentCycleID}"
    response = requests.get(BASE_GRADES_URL,verify=False)
    if response.status_code == 200:
        return response.json()
    print(response.status_code)
    return None

In [3]:
import requests
import pandas as pd
import time
from random import randint
import json



In [36]:
#Requesters
# Function to handle retries for API requests
def make_request_with_retry(url, max_retries=5, delay=1, backoff_factor=2):
    retries = 0
    gotResponse = False
    while not gotResponse:
        response = requests.get(url, verify=False)
        if response.status_code == 200:
            gotResponse = True
            return response.json()
        elif response.status_code == 503:  # Service Unavailable
            retries += 1
            wait_time = randint(0, 3)  # Add randomness for backoff
            time.sleep(wait_time)
        else:
            print(f"Error {response.status_code} for URL {url}. Retrying...")
            time.sleep(delay)
    return None

# Fetch cycles data with retry logic
def get_cycles_data(curp):
    BASE_CURP_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/schoolCyclesByStudent/curp={curp}"
    return make_request_with_retry(BASE_CURP_URL)

# Fetch grades data with retry logic
def get_grades_data(StudentCycleID):
    BASE_GRADES_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/evaluationsByStudentCycle/idStudentCycle={StudentCycleID}"
    return make_request_with_retry(BASE_GRADES_URL)

In [41]:
#Processor

# Fetch student data and create DataFrame
def fetch_student_data(curp):
    # Get school cycle data
    cycles_data = get_cycles_data(curp)

    if cycles_data is None or 'students' not in cycles_data:
        print(f"No student data found for CURP: {curp}")
        return None
    
    student = cycles_data['students'][0]
    student_id = student['id']
    CURP = student['curp']
    student_name = student['name']

    
    # Prepare a list to collect the data
    data = []

    
    
    # Extract each cycle information
    for cycle in student['schoolCyclesDTO']:

        
        StudentCycleID = cycle['idStudentCycle']
        level = cycle['level']
        grade = cycle['grade']
        school_cycle = cycle['schoolCycle']
        shift = cycle['shift']
        avg = cycle['avgAcreditation']
        avg = avg if avg != 0.0 else 'N/A'
        grado = cycle['grade']
        grupo = cycle['group']

        data.append({
        'CURP': curp,
        'Nombre': student_name,
        'Nivel': level,
        'CicloEscolar': school_cycle,
        'Materia': 'Promedio',
        'Calificación': avg,
        'Periodo': 'N/A',
        'Turno': shift,
        'Grado': grado,
        'Grupo': grupo
    })
        
        # Get grades for each cycle
        grades_data = get_grades_data(StudentCycleID)
  
        if grades_data is None or 'evaluationStudentCycle' not in grades_data:
            print(f"No grade data found for StudentCycleID: {StudentCycleID}")
            continue
        
        # Loop through the periods and subjects
        cycles =  grades_data['evaluationStudentCycle']
        if len(cycles) == 0:
            print(f"No grade data found for StudentCycleID: {StudentCycleID}")
            continue
        
        periodos = cycles[0]['evaluationPeriodsDTO']
        for periodo in periodos:
            period = periodo['period']
            for subject_info in periodo['subjectsAssesedDTO']:
                subject_name = subject_info['subject']
                grade = subject_info['subjectGrade']
                
                # Collect relevant data
                data.append({
                    'CURP': curp,
                    'Nombre': student_name,
                    'Nivel': level,
                    'CicloEscolar': school_cycle,
                    'Materia': subject_name,
                    'Calificación': grade,
                    'Periodo': period,
                    'Turno': shift,
                    'Grado': grado,
                    'Grupo': grupo
                })
        
        # Add a row for the average grade
    

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

In [42]:
fetch_student_data("MEAJ030506MHGJRQA4")


/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


No grade data found for StudentCycleID: 118482038


/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


No grade data found for StudentCycleID: 94235037


/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,CURP,Nombre,Nivel,CicloEscolar,Materia,Calificación,Periodo,Turno,Grado,Grupo
0,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2015-2016,Promedio,7.8,N/A,MATUTINO,PRIMERO,D
1,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2016-2017,Promedio,8.4,N/A,MATUTINO,SEGUNDO,D
2,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Promedio,N/A,N/A,MATUTINO,TERCERO,D
3,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,"Artes III (Música, Danza, Teatro o Artes visua...",10.0,Septiembre - Octubre,MATUTINO,TERCERO,D
4,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Ciencias III (Enfasis en quimica),8.5,Septiembre - Octubre,MATUTINO,TERCERO,D
5,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Educacion fisica III,10.0,Septiembre - Octubre,MATUTINO,TERCERO,D
6,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Español III,8.8,Septiembre - Octubre,MATUTINO,TERCERO,D
7,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Formacion civica y etica II,10.0,Septiembre - Octubre,MATUTINO,TERCERO,D
8,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Historia II,10.0,Septiembre - Octubre,MATUTINO,TERCERO,D
9,MEAJ030506MHGJRQA4,JAQUELINE,SECUNDARIA,2017-2018,Matemáticas III,7.0,Septiembre - Octubre,MATUTINO,TERCERO,D


In [16]:
get_cycles_data("MEAJ030506MHGJRQA4")

/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'students': [{'id': 13277356,
   'curp': 'MEAJ030506MHGJRQA4',
   'name': 'JAQUELINE',
   'schoolCyclesDTO': [{'idStudentCycle': 118482038,
     'idSchoolCycle': 2,
     'idGrade': 1,
     'level': 'SECUNDARIA',
     'grade': 'PRIMERO',
     'schoolCycle': '2015-2016',
     'group': 'D',
     'shift': 'MATUTINO',
     'lastCycle': 0,
     'avgAcreditation': 7.8,
     'updatedAt': '17/02/2023'},
    {'idStudentCycle': 94235037,
     'idSchoolCycle': 3,
     'idGrade': 2,
     'level': 'SECUNDARIA',
     'grade': 'SEGUNDO',
     'schoolCycle': '2016-2017',
     'group': 'D',
     'shift': 'MATUTINO',
     'lastCycle': 0,
     'avgAcreditation': 8.4,
     'updatedAt': '18/02/2023'},
    {'idStudentCycle': 7881592,
     'idSchoolCycle': 4,
     'idGrade': 3,
     'level': 'SECUNDARIA',
     'grade': 'TERCERO',
     'schoolCycle': '2017-2018',
     'group': 'D',
     'shift': 'MATUTINO',
     'lastCycle': 1,
     'avgAcreditation': 0.0,
     'updatedAt': '22/08/2018'}]}]}

In [ ]:
# Example usage:
curp = "CUPM960602MCSRRN08"  # Replace with the CURP you want to test
student_data_df = fetch_student_data(curp)
print(student_data_df)

No student data found for CURP: AAAA000102MVZLLTA6
None


/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [48]:
student_data_df.to_csv("student_data2.csv", index=False)

In [9]:
curps = pd.read_csv("~/Downloads/CURPS.csv")

In [14]:
curps.sample(10)

,curp
12438525,HEHS890328MVZRRL04
10496272,GOGX011110MSPNLMA5
11362562,GUGG950923MMNZRL69
13037270,HESD970817MDFRNN01
9477162,GAMS900929MJCLNR07
3197469,BUBP920625MDFZRL04
23289348,RASN021101MCSBLLA3
17505218,MEAJ030506MHGJRQA4
8473831,FORS900504MVZLYL01
7558641,EIMS901113MSLNDR03
